In [ ]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
import numpy as np
import cv2
from scipy.optimize import linear_sum_assignment
from filterpy.kalman import KalmanFilter
import matplotlib.pyplot as plt
from pathlib import Path
import time
from collections import defaultdict
from IPython.display import clear_output
from google.colab import drive
import os

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

PyTorch: 2.8.0+cu126
CUDA: True


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 디렉터리 구조

In [ ]:
def check_mot16_structure():
    print("\n" + "="*60)
    print("Checking MOT16 Dataset Structure")
    print("="*60)

    base_paths = [
        '/content/MOT16',
        '/content/drive/MyDrive/MOT16',
        '/content/mot16',
        '/content/drive/MyDrive/mot16'
    ]

    mot16_path = None
    for path in base_paths:
        if os.path.exists(path):
            mot16_path = path
            print(f"\nFound MOT16 at: {path}")
            break

    if mot16_path is None:
        print("\nMOT16 not found. Please specify the correct path.")
        print("Expected structure:")
        print("MOT16/")
        print("  train/")
        print("    MOT16-02/")
        print("    MOT16-04/")
        print("    ...")
        return None

    train_path = os.path.join(mot16_path, 'train')
    if not os.path.exists(train_path):
        print(f"\nTrain folder not found at: {train_path}")
        print("\nAvailable folders:")
        for item in os.listdir(mot16_path):
            print(f"  - {item}")
        return None

    print(f"\nTrain path: {train_path}")
    print("\nAvailable sequences:")
    sequences = sorted([d for d in os.listdir(train_path)
                       if os.path.isdir(os.path.join(train_path, d))])

    for seq in sequences:
        seq_path = os.path.join(train_path, seq)
        print(f"\n{seq}/")

        for item in sorted(os.listdir(seq_path)):
            item_path = os.path.join(seq_path, item)
            if os.path.isdir(item_path):
                num_files = len(os.listdir(item_path))
                print(f"  - {item}/ ({num_files} files)")
            else:
                print(f"  - {item}")

    return train_path

MOT16_PATH = check_mot16_structure()

if MOT16_PATH is None:
    print("\nPlease update MOT16_PATH manually:")
    print("MOT16_PATH = '/your/correct/path/to/MOT16/train'")
else:
    print(f"\nMOT16_PATH set to: {MOT16_PATH}")


Checking MOT16 Dataset Structure

Found MOT16 at: /content/drive/MyDrive/MOT16

Train path: /content/drive/MyDrive/MOT16/train

Available sequences:

MOT16-02/
  - det/ (1 files)
  - gt/ (1 files)
  - img1/ (600 files)
  - seqinfo.ini

MOT16-04/
  - det/ (1 files)
  - gt/ (1 files)
  - img1/ (1050 files)
  - seqinfo.ini

MOT16-05/
  - det/ (1 files)
  - gt/ (1 files)
  - img1/ (837 files)
  - seqinfo.ini

MOT16-09/
  - det/ (1 files)
  - gt/ (1 files)
  - img1/ (525 files)
  - seqinfo.ini

MOT16-10/
  - det/ (1 files)
  - gt/ (1 files)
  - img1/ (654 files)
  - seqinfo.ini

MOT16-11/
  - det/ (1 files)
  - gt/ (1 files)
  - img1/ (900 files)
  - seqinfo.ini

MOT16-13/
  - det/ (1 files)
  - gt/ (1 files)
  - img1/ (750 files)
  - seqinfo.ini

MOT16_PATH set to: /content/drive/MyDrive/MOT16/train


# 칼만 필터

In [ ]:
class KalmanBoxTracker:
    count = 0

    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)

        self.kf.F = np.array([
            [1,0,0,0,1,0,0],
            [0,1,0,0,0,1,0],
            [0,0,1,0,0,0,1],
            [0,0,0,1,0,0,0],
            [0,0,0,0,1,0,0],
            [0,0,0,0,0,1,0],
            [0,0,0,0,0,0,1]
        ])

        self.kf.H = np.array([
            [1,0,0,0,0,0,0],
            [0,1,0,0,0,0,0],
            [0,0,1,0,0,0,0],
            [0,0,0,1,0,0,0]
        ])

        self.kf.R[2:,2:] *= 10.0
        self.kf.P[4:,4:] *= 1000.0
        self.kf.P *= 10.0
        self.kf.Q[-1,-1] *= 0.01
        self.kf.Q[4:,4:] *= 0.01

        self.kf.x[:4] = self.convert_bbox_to_z(bbox)

        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []
        self.hits = 0
        self.hit_streak = 0
        self.age = 0

    def update(self, bbox):
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1
        self.kf.update(self.convert_bbox_to_z(bbox))

    def predict(self):
        if self.kf.x[6] + self.kf.x[2] <= 0:
            self.kf.x[6] *= 0.0

        self.kf.predict()
        self.age += 1

        if self.time_since_update > 0:
            self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(self.convert_x_to_bbox(self.kf.x))
        return self.history[-1]

    def get_state(self):
        return self.convert_x_to_bbox(self.kf.x)

    @staticmethod
    def convert_bbox_to_z(bbox):
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        u = bbox[0] + w/2.0
        v = bbox[1] + h/2.0
        s = w * h
        r = w / float(h)
        return np.array([u, v, s, r]).reshape((4, 1))

    @staticmethod
    def convert_x_to_bbox(x, score=None):
        w = np.sqrt(x[2] * x[3])
        h = x[2] / w
        if score is None:
            return np.array([
                x[0] - w/2.0,
                x[1] - h/2.0,
                x[0] + w/2.0,
                x[1] + h/2.0
            ]).reshape((1, 4))
        else:
            return np.array([
                x[0] - w/2.0,
                x[1] - h/2.0,
                x[0] + w/2.0,
                x[1] + h/2.0,
                score
            ]).reshape((1, 5))


# data associate

In [ ]:
def iou_batch(bboxes1, bboxes2):
    bboxes2 = np.expand_dims(bboxes2, 0)
    bboxes1 = np.expand_dims(bboxes1, 1)

    xx1 = np.maximum(bboxes1[..., 0], bboxes2[..., 0])
    yy1 = np.maximum(bboxes1[..., 1], bboxes2[..., 1])
    xx2 = np.minimum(bboxes1[..., 2], bboxes2[..., 2])
    yy2 = np.minimum(bboxes1[..., 3], bboxes2[..., 3])

    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)

    intersection = w * h
    area1 = (bboxes1[..., 2] - bboxes1[..., 0]) * (bboxes1[..., 3] - bboxes1[..., 1])
    area2 = (bboxes2[..., 2] - bboxes2[..., 0]) * (bboxes2[..., 3] - bboxes2[..., 1])
    union = area1 + area2 - intersection

    iou = intersection / union
    return iou


In [ ]:
def associate_detections_to_trackers(detections, trackers, iou_threshold=0.3):
    if len(trackers) == 0:
        return np.empty((0, 2), dtype=int), np.arange(len(detections)), np.empty((0,), dtype=int)

    iou_matrix = iou_batch(detections, trackers)

    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            row_ind, col_ind = linear_sum_assignment(-iou_matrix)
            matched_indices = np.stack([row_ind, col_ind], axis=1)
    else:
        matched_indices = np.empty((0, 2), dtype=int)

    unmatched_detections = []
    for d in range(len(detections)):
        if d not in matched_indices[:, 0]:
            unmatched_detections.append(d)

    unmatched_trackers = []
    for t in range(len(trackers)):
        if t not in matched_indices[:, 1]:
            unmatched_trackers.append(t)

    matches = []
    for m in matched_indices:
        if iou_matrix[m[0], m[1]] < iou_threshold:
            unmatched_detections.append(m[0])
            unmatched_trackers.append(m[1])
        else:
            matches.append(m.reshape(1, 2))

    if len(matches) == 0:
        matches = np.empty((0, 2), dtype=int)
    else:
        matches = np.concatenate(matches, axis=0)

    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)

# SORT

In [ ]:
class Sort:
    def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers = []
        self.frame_count = 0

    def update(self, detections):
        self.frame_count += 1

        trks = np.zeros((len(self.trackers), 5))
        to_del = []
        ret = []

        for t, trk in enumerate(trks):
            pos = self.trackers[t].predict()[0]
            trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
            if np.any(np.isnan(pos)):
                to_del.append(t)

        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
        for t in reversed(to_del):
            self.trackers.pop(t)

        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(
            detections[:, :4] if len(detections) > 0 else np.empty((0, 4)),
            trks[:, :4],
            self.iou_threshold
        )

        for m in matched:
            self.trackers[m[1]].update(detections[m[0], :4])

        for i in unmatched_dets:
            trk = KalmanBoxTracker(detections[i, :4])
            self.trackers.append(trk)

        i = len(self.trackers)
        for trk in reversed(self.trackers):
            d = trk.get_state()[0]

            if (trk.time_since_update < 1) and \
               (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
                ret.append(np.concatenate((d, [trk.id + 1])).reshape(1, -1))

            i -= 1
            if trk.time_since_update > self.max_age:
                self.trackers.pop(i)

        if len(ret) > 0:
            return np.concatenate(ret)
        return np.empty((0, 5))

# Detector

In [ ]:
class FasterRCNNDetector:
    def __init__(self, confidence_threshold=0.5, device='cuda'):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        print(f"Device: {self.device}")

        self.model = fasterrcnn_resnet50_fpn(pretrained=True)
        self.model.to(self.device)
        self.model.eval()

        self.confidence_threshold = confidence_threshold
        self.person_class_id = 1

    def detect(self, image):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_tensor = F.to_tensor(image_rgb).to(self.device)

        with torch.no_grad():
            predictions = self.model([image_tensor])[0]

        person_mask = predictions['labels'] == self.person_class_id
        boxes = predictions['boxes'][person_mask].cpu().numpy()
        scores = predictions['scores'][person_mask].cpu().numpy()

        high_conf_mask = scores >= self.confidence_threshold
        boxes = boxes[high_conf_mask]
        scores = scores[high_conf_mask]

        if len(boxes) == 0:
            return np.empty((0, 5))

        detections = np.column_stack([boxes, scores])
        return detections


# DataLoader

In [ ]:
class MOTDataLoader:
    def __init__(self, dataset_path, sequence_name):
        self.dataset_path = Path(dataset_path)
        self.sequence_name = sequence_name
        self.sequence_path = self.dataset_path / sequence_name

        self.img_path = self.sequence_path / 'img1'
        self.images = sorted(self.img_path.glob('*.jpg'))

        self.gt_path = self.sequence_path / 'gt' / 'gt.txt'
        self.gt_data = self.load_ground_truth() if self.gt_path.exists() else None

        print(f"Sequence: {sequence_name}")
        print(f"Frames: {len(self.images)}")

    def load_ground_truth(self):
        gt_data = defaultdict(list)
        with open(self.gt_path, 'r') as f:
            for line in f:
                parts = line.strip().split(',')
                frame_id = int(parts[0])
                track_id = int(parts[1])
                bbox = [float(x) for x in parts[2:6]]
                confidence = float(parts[6])
                class_id = int(parts[7])

                if confidence == 1 and class_id == 1:
                    bbox_xyxy = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
                    gt_data[frame_id].append({
                        'track_id': track_id,
                        'bbox': bbox_xyxy
                    })
        return gt_data

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        frame_id = int(img_path.stem)
        image = cv2.imread(str(img_path))

        gt = self.gt_data[frame_id] if self.gt_data else None
        return frame_id, image, gt

# MOTMetrics

In [ ]:
class MOTMetrics:
    def __init__(self):
        self.reset()

    def reset(self):
        self.num_frames = 0
        self.num_matches = 0
        self.num_false_positives = 0
        self.num_misses = 0
        self.num_switches = 0
        self.last_match = {}

    def update(self, gt_boxes, pred_boxes, iou_threshold=0.5):
        self.num_frames += 1

        if len(gt_boxes) == 0 and len(pred_boxes) == 0:
            return

        if len(pred_boxes) == 0:
            self.num_misses += len(gt_boxes)
            return

        if len(gt_boxes) == 0:
            self.num_false_positives += len(pred_boxes)
            return

        gt_bboxes = np.array([gt['bbox'] for gt in gt_boxes])
        pred_bboxes = pred_boxes[:, :4]

        iou_matrix = iou_batch(gt_bboxes, pred_bboxes)

        row_ind, col_ind = linear_sum_assignment(-iou_matrix)

        matched_gt = set()
        matched_pred = set()

        for gt_idx, pred_idx in zip(row_ind, col_ind):
            if iou_matrix[gt_idx, pred_idx] >= iou_threshold:
                self.num_matches += 1
                matched_gt.add(gt_idx)
                matched_pred.add(pred_idx)

                gt_id = gt_boxes[gt_idx]['track_id']
                pred_id = int(pred_boxes[pred_idx, 4])

                if pred_id in self.last_match:
                    if self.last_match[pred_id] != gt_id:
                        self.num_switches += 1

                self.last_match[pred_id] = gt_id

        self.num_false_positives += len(pred_boxes) - len(matched_pred)
        self.num_misses += len(gt_boxes) - len(matched_gt)

    def get_metrics(self):
        total_gt = self.num_matches + self.num_misses

        if total_gt == 0:
            return {}

        mota = 1 - (self.num_misses + self.num_false_positives + self.num_switches) / total_gt

        total_pred = self.num_matches + self.num_false_positives
        precision = self.num_matches / total_pred if total_pred > 0 else 0
        recall = self.num_matches / total_gt if total_gt > 0 else 0

        return {
            'MOTA': mota * 100,
            'Precision': precision * 100,
            'Recall': recall * 100,
            'FP': self.num_false_positives,
            'FN': self.num_misses,
            'ID_switches': self.num_switches,
            'Num_frames': self.num_frames
        }

# 시각화

In [ ]:
def draw_tracks(image, tracks, frame_id=None, fps=None):
    image_copy = image.copy()

    for track in tracks:
        x1, y1, x2, y2, track_id = track
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        track_id = int(track_id)

        color_id = plt.cm.tab20(track_id % 20)[:3]
        color_bgr = tuple([int(c * 255) for c in color_id])

        cv2.rectangle(image_copy, (x1, y1), (x2, y2), color_bgr, 3)

        label = f"ID: {track_id}"
        label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        cv2.rectangle(image_copy, (x1, y1-label_size[1]-10),
                     (x1+label_size[0]+10, y1), color_bgr, -1)
        cv2.putText(image_copy, label, (x1+5, y1-5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    panel_height = 80
    overlay = image_copy.copy()
    cv2.rectangle(overlay, (0, 0), (image_copy.shape[1], panel_height), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.6, image_copy, 0.4, 0, image_copy)

    if frame_id is not None:
        cv2.putText(image_copy, f"Frame: {frame_id}", (15, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.putText(image_copy, f"Tracks: {len(tracks)}", (15, 60),
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    if fps is not None:
        cv2.putText(image_copy, f"FPS: {fps:.1f}", (image_copy.shape[1]-150, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    return image_copy

# 실행 함수

In [ ]:
def run_sort_tracker(sequence_name='MOT16-02', max_frames=None,
                     visualize=True, save_video=True,
                     output_dir='/content'):
    print("="*60)
    print(f"Running SORT on {sequence_name}")
    print("="*60)

    data_loader = MOTDataLoader(MOT16_PATH, sequence_name)

    print("\nInitializing detector...")
    detector = FasterRCNNDetector(confidence_threshold=0.5)

    print("Initializing SORT tracker...")
    print("  max_age: 1")
    print("  min_hits: 3")
    print("  iou_threshold: 0.3")
    tracker = Sort(max_age=1, min_hits=3, iou_threshold=0.3)

    metrics = MOTMetrics()

    video_writer = None
    output_video_path = None

    if save_video:
        first_frame = cv2.imread(str(data_loader.images[0]))
        h, w = first_frame.shape[:2]

        output_video_path = f'{output_dir}/{sequence_name}_tracking.mp4'

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_video_path, fourcc, 20.0, (w, h))

        if video_writer.isOpened():
            print(f"Video output: {output_video_path}")
        else:
            print(f"Failed to create video writer")
            video_writer = None

    print("\n" + "="*60)
    print("Processing frames...")
    print("="*60)

    num_frames = min(len(data_loader), max_frames) if max_frames else len(data_loader)
    total_time = 0
    detection_time = 0
    tracking_time = 0

    all_tracks = []

    for idx in range(num_frames):
        frame_id, image, gt_boxes = data_loader[idx]

        det_start = time.time()
        detections = detector.detect(image)
        detection_time += time.time() - det_start

        track_start = time.time()
        tracks = tracker.update(detections)
        tracking_time += time.time() - track_start

        frame_time = time.time() - det_start
        total_time += frame_time

        for track in tracks:
            all_tracks.append([frame_id] + track.tolist())

        if gt_boxes is not None:
            metrics.update(gt_boxes, tracks)

        if (idx + 1) % 10 == 0 or idx == 0:
            avg_fps = (idx + 1) / total_time if total_time > 0 else 0
            print(f"Frame {idx + 1}/{num_frames} | "
                  f"Det: {len(detections):2d} | "
                  f"Trk: {len(tracks):2d} | "
                  f"FPS: {avg_fps:.1f}")

        if visualize and idx % 10 == 0:
            vis_frame = draw_tracks(image, tracks, frame_id, (idx+1)/total_time)

            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(vis_frame, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.title(f"{sequence_name} - Frame {frame_id}")
            plt.tight_layout()
            plt.show()
            clear_output(wait=True)

        if save_video and video_writer and video_writer.isOpened():
            vis_frame = draw_tracks(image, tracks, frame_id, (idx+1)/total_time)
            video_writer.write(vis_frame)

    print("\n" + "="*60)
    print("COMPLETED")
    print("="*60)

    avg_fps = num_frames / total_time if total_time > 0 else 0
    det_fps = num_frames / detection_time if detection_time > 0 else 0
    track_fps = num_frames / tracking_time if tracking_time > 0 else 0

    print(f"\nPerformance:")
    print(f"  Frames: {num_frames}")
    print(f"  Total time: {total_time:.2f}s")
    print(f"  Overall FPS: {avg_fps:.2f}")
    print(f"  Detection FPS: {det_fps:.2f}")
    print(f"  Tracking FPS: {track_fps:.2f}")

    if data_loader.gt_data is not None:
        results = metrics.get_metrics()
        print(f"\nMetrics:")
        print(f"  MOTA: {results['MOTA']:.2f}%")
        print(f"  Precision: {results['Precision']:.2f}%")
        print(f"  Recall: {results['Recall']:.2f}%")
        print(f"  FP: {results['FP']}")
        print(f"  FN: {results['FN']}")
        print(f"  ID switches: {results['ID_switches']}")

    output_txt = f'{output_dir}/{sequence_name}_result.txt'
    with open(output_txt, 'w') as f:
        for track_data in all_tracks:
            f.write(','.join(map(str, track_data)) + '\n')
    print(f"\nResults saved: {output_txt}")

    if video_writer and video_writer.isOpened():
        video_writer.release()
        print(f"\n{'='*60}")
        print(f"Video saved successfully!")
        print(f"  Path: {output_video_path}")
        print(f"  Frames: {num_frames}")
        print(f"  Resolution: {w}x{h}")
        print(f"  FPS: 20.0")
        print(f"{'='*60}")

        try:
            from IPython.display import FileLink
            display(FileLink(output_video_path))
        except:
            pass

    return tracker, metrics, all_tracks


print("\n" + "="*60)
print("SORT Tracker Ready")
print("="*60)
print("\nRun:")
print("tracker, metrics, results = run_sort_tracker('MOT16-02', max_frames=100)")

if MOT16_PATH is not None:
    sequences = ['MOT16-02', 'MOT16-04', 'MOT16-05', 'MOT16-09', 'MOT16-10', 'MOT16-11', 'MOT16-13']

    for seq in sequences:
        print(f"\n\nProcessing {seq}...")
        tracker, metrics, results = run_sort_tracker(
            sequence_name=seq,
            max_frames=None,
            visualize=False,
            save_video=True
        )
else:
    print("Please set MOT16_PATH first")


SORT Tracker Ready

Run:
tracker, metrics, results = run_sort_tracker('MOT16-02', max_frames=100)


Processing MOT16-02...
Running SORT on MOT16-02
Sequence: MOT16-02
Frames: 600

Initializing detector...
Device: cuda
Initializing SORT tracker...
  max_age: 1
  min_hits: 3
  iou_threshold: 0.3
Video output: /content/MOT16-02_tracking.mp4

Processing frames...
Frame 1/600 | Det: 26 | Trk: 26 | FPS: 5.7
Frame 10/600 | Det: 23 | Trk: 20 | FPS: 7.2
Frame 20/600 | Det: 24 | Trk: 21 | FPS: 7.3
Frame 30/600 | Det: 27 | Trk: 20 | FPS: 7.3
Frame 40/600 | Det: 23 | Trk: 20 | FPS: 7.3
Frame 50/600 | Det: 26 | Trk: 24 | FPS: 7.2
Frame 60/600 | Det: 32 | Trk: 22 | FPS: 7.2
Frame 70/600 | Det: 23 | Trk: 20 | FPS: 7.2
Frame 80/600 | Det: 24 | Trk: 16 | FPS: 7.2
Frame 90/600 | Det: 18 | Trk: 16 | FPS: 7.2
Frame 100/600 | Det: 23 | Trk: 19 | FPS: 7.2
Frame 110/600 | Det: 23 | Trk: 18 | FPS: 7.2
Frame 120/600 | Det: 22 | Trk: 17 | FPS: 7.2
Frame 130/600 | Det: 22 | Trk: 18 | FPS: 7.2
Frame 140/600 | De

/content/MOT16-02_tracking.mp4



Processing MOT16-04...
Running SORT on MOT16-04
Sequence: MOT16-04
Frames: 1050

Initializing detector...
Device: cuda
Initializing SORT tracker...
  max_age: 1
  min_hits: 3
  iou_threshold: 0.3
Video output: /content/MOT16-04_tracking.mp4

Processing frames...
Frame 1/1050 | Det: 41 | Trk: 41 | FPS: 7.1
Frame 10/1050 | Det: 40 | Trk: 33 | FPS: 7.3
Frame 20/1050 | Det: 36 | Trk: 34 | FPS: 7.1
Frame 30/1050 | Det: 38 | Trk: 36 | FPS: 7.2
Frame 40/1050 | Det: 36 | Trk: 34 | FPS: 7.3
Frame 50/1050 | Det: 35 | Trk: 33 | FPS: 7.3
Frame 60/1050 | Det: 35 | Trk: 34 | FPS: 7.3
Frame 70/1050 | Det: 36 | Trk: 34 | FPS: 7.3
Frame 80/1050 | Det: 38 | Trk: 35 | FPS: 7.3
Frame 90/1050 | Det: 37 | Trk: 33 | FPS: 7.4
Frame 100/1050 | Det: 39 | Trk: 34 | FPS: 7.4
Frame 110/1050 | Det: 37 | Trk: 35 | FPS: 7.4
Frame 120/1050 | Det: 35 | Trk: 32 | FPS: 7.4
Frame 130/1050 | Det: 32 | Trk: 31 | FPS: 7.4
Frame 140/1050 | Det: 38 | Trk: 33 | FPS: 7.4
Frame 150/1050 | Det: 37 | Trk: 33 | FPS: 7.4
Frame 160/

/content/MOT16-04_tracking.mp4



Processing MOT16-05...
Running SORT on MOT16-05
Sequence: MOT16-05
Frames: 837

Initializing detector...
Device: cuda
Initializing SORT tracker...
  max_age: 1
  min_hits: 3
  iou_threshold: 0.3
Video output: /content/MOT16-05_tracking.mp4

Processing frames...
Frame 1/837 | Det:  2 | Trk:  2 | FPS: 9.3
Frame 10/837 | Det: 11 | Trk:  6 | FPS: 8.7
Frame 20/837 | Det:  9 | Trk:  5 | FPS: 8.9
Frame 30/837 | Det:  7 | Trk:  4 | FPS: 9.0
Frame 40/837 | Det: 13 | Trk: 11 | FPS: 9.0
Frame 50/837 | Det: 11 | Trk:  5 | FPS: 9.0
Frame 60/837 | Det:  6 | Trk:  5 | FPS: 9.1
Frame 70/837 | Det: 12 | Trk:  7 | FPS: 9.1
Frame 80/837 | Det:  6 | Trk:  6 | FPS: 9.1
Frame 90/837 | Det: 10 | Trk:  8 | FPS: 9.1
Frame 100/837 | Det: 14 | Trk: 11 | FPS: 9.1
Frame 110/837 | Det:  8 | Trk:  7 | FPS: 9.1
Frame 120/837 | Det: 12 | Trk:  4 | FPS: 9.1
Frame 130/837 | Det: 14 | Trk: 10 | FPS: 9.0
Frame 140/837 | Det: 12 | Trk: 11 | FPS: 9.0
Frame 150/837 | Det:  9 | Trk:  7 | FPS: 9.0
Frame 160/837 | Det: 14 | T

...

/content/MOT16-10_tracking.mp4



Processing MOT16-11...
Running SORT on MOT16-11
Sequence: MOT16-11
Frames: 900

Initializing detector...
Device: cuda
Initializing SORT tracker...
  max_age: 1
  min_hits: 3
  iou_threshold: 0.3
Video output: /content/MOT16-11_tracking.mp4

Processing frames...
Frame 1/900 | Det: 17 | Trk: 17 | FPS: 7.5
Frame 10/900 | Det: 20 | Trk: 14 | FPS: 7.6
Frame 20/900 | Det: 14 | Trk: 11 | FPS: 7.4
Frame 30/900 | Det: 11 | Trk:  9 | FPS: 7.4
Frame 40/900 | Det: 11 | Trk: 10 | FPS: 7.5
Frame 50/900 | Det: 13 | Trk: 11 | FPS: 7.5
Frame 60/900 | Det: 14 | Trk: 11 | FPS: 7.5
Frame 70/900 | Det: 15 | Trk: 12 | FPS: 7.5
Frame 80/900 | Det: 22 | Trk: 14 | FPS: 7.5
Frame 90/900 | Det: 20 | Trk: 12 | FPS: 7.5
Frame 100/900 | Det: 16 | Trk: 13 | FPS: 7.5
Frame 110/900 | Det: 14 | Trk:  9 | FPS: 7.5
Frame 120/900 | Det:  8 | Trk:  7 | FPS: 7.5
Frame 130/900 | Det:  9 | Trk:  8 | FPS: 7.5
Frame 140/900 | Det:  9 | Trk:  8 | FPS: 7.5
Frame 150/900 | Det: 11 | Trk:  9 | FPS: 7.5
Frame 160/900 | Det:  9 | T

/content/MOT16-11_tracking.mp4



Processing MOT16-13...
Running SORT on MOT16-13
Sequence: MOT16-13
Frames: 750

Initializing detector...
Device: cuda
Initializing SORT tracker...
  max_age: 1
  min_hits: 3
  iou_threshold: 0.3
Video output: /content/MOT16-13_tracking.mp4

Processing frames...
Frame 1/750 | Det: 23 | Trk: 23 | FPS: 7.5
Frame 10/750 | Det: 24 | Trk: 21 | FPS: 7.1
Frame 20/750 | Det: 20 | Trk: 14 | FPS: 7.3
Frame 30/750 | Det: 17 | Trk: 14 | FPS: 7.4
Frame 40/750 | Det: 15 | Trk: 13 | FPS: 7.5
Frame 50/750 | Det: 22 | Trk: 19 | FPS: 7.5
Frame 60/750 | Det: 22 | Trk: 18 | FPS: 7.5
Frame 70/750 | Det: 21 | Trk: 16 | FPS: 7.5
Frame 80/750 | Det: 26 | Trk: 19 | FPS: 7.5
Frame 90/750 | Det: 31 | Trk: 20 | FPS: 7.5
Frame 100/750 | Det: 25 | Trk: 20 | FPS: 7.6
Frame 110/750 | Det: 27 | Trk: 21 | FPS: 7.6
Frame 120/750 | Det: 22 | Trk: 18 | FPS: 7.6
Frame 130/750 | Det: 22 | Trk: 14 | FPS: 7.6
Frame 140/750 | Det: 16 | Trk: 13 | FPS: 7.6
Frame 150/750 | Det: 18 | Trk: 13 | FPS: 7.5
Frame 160/750 | Det: 11 | T

/content/MOT16-13_tracking.mp4